In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 206.0 MB/s eta 0:00:0000:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9


In [9]:
# Prepare some Countdown data

from datasets import load_dataset

ds = load_dataset("Jiayi-Pan/Countdown-Tasks-3to4")

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['target', 'nums'],
        num_rows: 490364
    })
})

In [11]:

# f"Using the numbers {nums}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer>\\boxed{{(1 + 2) / 3}}</answer>. Think step by step inside <think> tags."


In [12]:
dataset_list = [
    [
        {
            "from": "human",
            "value": (
                f"Using the numbers {row['nums']}, create an equation that equals {row['target']}. "
                "You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once. "
                "Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, "
                "for example <answer> (1 + 2) / 3 </answer>. Think step by step inside <think> tags."
            )
        },
        {
            "from": "assistant",
            "ground_truth": {
                "value": f"{row['target']}"
            },
            "nums": {
                "value": f"{row['nums']}"
            }
        }
    ]
    for row in ds["train"]
]


In [13]:
print(dataset_list[0][0]['value'])

Using the numbers [44, 19, 35], create an equation that equals 98. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>. Think step by step inside <think> tags.


In [15]:
import json

with open('/data/users/kevihuang/projects/Open-Reasoner-Zero/data/countdown-tasks-3to4_first_5000.json', 'w') as f:
    json.dump(dataset_list[:5000], f, indent=2)

# Prepare Query Data

In [3]:
# Prepare some Countdown data

from datasets import load_dataset

ds = load_dataset("khuang2/math-query-gen-prompts-w-solution") #khuang2/math-query-gen-prompts") #"khuang2/Countdown-Tasks-3to4-query-gen-prompts")

/data/users/kevihuang/projects/Open-Reasoner-Zero/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 50000/50000 [00:00<00:00, 315059.41 examples/s]


In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 50000
    })
})

In [5]:
ds['train'][0]

{'prompt': "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nGiven the assistant goal, task generation success criteria, and example final answer, generate a task that fulfills the task generation success criteria. The generated task should be:\n\n1) maximizing user benefit and helpfulness and minimizing user risk, harm, and unwanted outcomes\n\n2) novel - avoid designing a task that is drawn from the same linguistic templates or differ only in superficial details\n\n3) difficult yet tractable - the task should be challenging for the assistant to complete, without being completely intractable\n\n4) clear - the task should be specific enough to not require further user input to understand what is being asked of the assistant and should provide all the context and detail necessary\n\nShow your work in <think> </think> tags. And return the task and final answer in <answer> </answer> tags.\n\n\nAssistant Goal: Get really 

In [6]:
prompt = ds['train'][0]['prompt']
print(prompt.split("<|im_start|>user")[1].split("<|im_end|>")[0].strip().replace("×", "*").replace("÷", "/"))

Given the assistant goal, task generation success criteria, and example final answer, generate a task that fulfills the task generation success criteria. The generated task should be:

1) maximizing user benefit and helpfulness and minimizing user risk, harm, and unwanted outcomes

2) novel - avoid designing a task that is drawn from the same linguistic templates or differ only in superficial details

3) difficult yet tractable - the task should be challenging for the assistant to complete, without being completely intractable

4) clear - the task should be specific enough to not require further user input to understand what is being asked of the assistant and should provide all the context and detail necessary

Show your work in <think> </think> tags. And return the task and final answer in <answer> </answer> tags.


Assistant Goal: Get really good at math.

Task Generation Success Criteria: The task and final answer must be a dictionary with 'prompt', 'target', and 'solution' keys. T

In [7]:
# countdown_prompt = '''Given the assistant goal, task generation success criteria, and example final answer, generate a task that fulfills the task generation success criteria. The generated task should be:

# 1) maximizing user benefit and helpfulness and minimizing user risk, harm, and unwanted outcomes

# 2) novel - avoid designing a task that is drawn from the same linguistic templates or differ only in superficial details

# 3) difficult yet tractable - the task should be challenging for the assistant to complete, without being completely intractable

# 4) clear - the task should be specific enough to not require further user input to understand what is being asked of the assistant and should provide all the context and detail necessary

# Show your work in <think> </think> tags. And return the task and final answer in <answer> </answer> tags.


# Assistant Goal: Get really good at the Countdown Game with 3 to 4 numbers. The Countdown game is a numbers puzzle where players use a set of randomly drawn numbers and basic arithmetic operations (+, -, *, /) to reach a target number.

# Task Generation Success Criteria: The task and final answer must be a dictionary with the 'target', 'nums', and 'prompt' keys. The 'target' key should have a value that is an integer, the 'nums' key should have a value that is a list of 3 or 4 integers, and the 'prompt' key should have a value that is a string in the same exact format as the example final answer prompt, but just with the nums and target updated as appropriate.

# Example Final Answer: <answer>
# {
#     'target': 44,
#     'nums': [6, 77, 73, 20],
#     'prompt': 'Using the numbers [6, 77, 73, 20], create an equation that equals 44. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>. Think step by step inside <think> tags.'
# }
# </answer>'''

# print(countdown_prompt)

In [8]:
dataset_list = [
    [
        {
            "from": "human",
            "value": prompt.split("<|im_start|>user")[1].split("<|im_end|>")[0].strip().replace("×", "*").replace("÷", "/").replace("Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>. Think step by step inside <think> tags.", "").replace("Show your work in <think> </think> tags. And return the task and final answer in <answer> </answer> tags.\n\n\n", "").replace("Example Final Answer: <answer>", "Example Reasoning and Final Answer Response Format: <think>\nreasoning process here\n</think>\n<answer>").replace("[6, 77, 73, 20]", "[1, 2, 3]").replace("44", "6") # having the prompt include the formatting is confusing for the query network when we're using a base model (formatting is always wrong) # countdown_prompt
        },
        {
            "from": "assistant",
            "ground_truth": {
                "value": ""
            }
        }
    ]
    for row in ds["train"]
]
len(dataset_list)

50000

In [9]:
print(dataset_list[0][0]['value'])

Given the assistant goal, task generation success criteria, and example final answer, generate a task that fulfills the task generation success criteria. The generated task should be:

1) maximizing user benefit and helpfulness and minimizing user risk, harm, and unwanted outcomes

2) novel - avoid designing a task that is drawn from the same linguistic templates or differ only in superficial details

3) difficult yet tractable - the task should be challenging for the assistant to complete, without being completely intractable

4) clear - the task should be specific enough to not require further user input to understand what is being asked of the assistant and should provide all the context and detail necessary

Assistant Goal: Get really good at math.

Task Generation Success Criteria: The task and final answer must be a dictionary with 'prompt', 'target', and 'solution' keys. The 'prompt' should be a math problem, the 'target' should be the expected answer to the math problem, and th

In [14]:
import json

with open('/data/users/kevihuang/projects/Open-Reasoner-Zero/data/math-query-gen-prompts_first_5000_w_solution.json', 'w') as f:
    json.dump(dataset_list[:5000], f, indent=2)

# Inspect Query Parsing

In [1]:
query_item = r'''{
    'prompt': "Find the equation of the curve (f(x)), which is the orthogonal trajectory of the given family of curves: x^2 + y^2 = 2cy",
    'target': 0.0,
    'solution': "Step 1: Find the slope of the given family of curves

Rewriting the equation of the family of curves:

\(y^2 - 2c y + x^2 = 0\)

Let \(y = f(x)\) and we're going to differentiate both sides with respect to x.

Step 2: Implicit differentiation

\(2y \frac{dy}{dx} - 2c + 2x \frac{dy}{dx} = 0\)

2y \frac{dy}{dx} + 2x \frac{dy}{dx} = 2c

\(\frac{dy}{dx} (2y + 2x) = 2c\)

Step 3: Find slope of orthogonal trajectories

The slope of the orthogonal trajectory is the negative reciprocal of the slope of the family of curves.

\(\frac{dy}{dx} = -\frac{1}{f'(x)} = -\frac{1}{\frac{2c}{2y + 2x}} = -\frac{y + x}{c}\)

Step 4: Derive the equation of the orthogonal trajectory

Using the slope found in step 3, we are going to substitute it into the equation \(y = f(x)\) to find the equation of the orthogonal trajectory.

Solution: \(x^2 + y^2 - 2c + 2xy = 0\)"
}'''

In [2]:
import re, ast

def fix_multiline_string(dict_str):
    # Match keys 'prompt' or 'solution' and capture their values
    pattern = r"('(?:prompt|solution)':\s*)(['\"])(.*?)(\2)"
    def replacer(match):
        start, content, end = match.groups()
        fixed_content = content.replace('\n', '\\n')
        return f"{start}{fixed_content}{end}"
    return re.sub(pattern, replacer, dict_str, flags=re.DOTALL)

def extract_answer_input(query_item):
    system_prefix = "A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\n"
    assistant_suffix = " Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.\nAssistant: <think>" # Give example final answer to help policy model understand output better

    # TODO: adjust these default values based on domain / dataset! Pull from example final answer from prompt
    default_prompt = "What is 1 + 2?"
    default_extras = {'target': 3, 'valid_prompt': False, 'answer': 3, 'diverse': False}
    required_keys = {"target", "prompt"}
    
    default_full_prompt = system_prefix + default_prompt + assistant_suffix
    default_valid_answer_input = (
        default_full_prompt,
        default_extras
    )
    
    try:
        dict_str = query_item            # Content inside <answer>...</answer>
        dict_str_fixed = fix_multiline_string(dict_str)
        answer_input = ast.literal_eval(dict_str_fixed)
        if not isinstance(answer_input, dict) or not required_keys.issubset(answer_input.keys()):
            return default_valid_answer_input
        prompt_text = str(answer_input.get('prompt')).strip()
        full_prompt = system_prefix + prompt_text + assistant_suffix
        answer_input['prompt'] = full_prompt
        answer_input['target'] = str(float(answer_input.get('target'))) # don't force to be an int unless for countdown
        answer_input['answer'] = answer_input['target']
        answer_input['nums'] = [int(x) for x in answer_input.get('nums', [])]

        answer_input['valid_prompt'] = True
        return (full_prompt, answer_input)
    except Exception as e:
        return default_valid_answer_input

In [ ]:
extract_answer_input(query_item)

('A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\nWhat is 1 + 2? Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.\nAssistant: <think>',
 {'target': 3, 'valid_prompt': False, 'answer': 3, 'diverse': False})

In [1]:
query_item = r'''{
  'prompt': "Calculate the sum of the first 200 prime numbers",
  'target': 2150,
  'solution': "The sum of the first 200 prime numbers is 2150."
}'''


import re, ast

def fix_multiline_string(dict_str):
    # Match keys 'prompt' or 'solution' and capture their values
    pattern = r"((?:'prompt'|\"prompt\"|'solution'|\"solution\")\s*:\s*)(['\"])(.*?)(\2)"
    def replacer(match):
        start, quote, content, _ = match.groups()
        fixed_content = content.replace('\n', '\\n')
        return f"{start}{quote}{fixed_content}{quote}"
    return re.sub(pattern, replacer, dict_str, flags=re.DOTALL)

def extract_answer_input(query_item):
    system_prefix = "A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\n"
    assistant_suffix = " Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.\nAssistant: <think>" # Give example final answer to help policy model understand output better

    # TODO: adjust these default values based on domain / dataset! Pull from example final answer from prompt
    default_prompt = "What is 1 + 2?"
    default_extras = {'target': 3, 'valid_prompt': False, 'answer': 3, 'diverse': False}
    required_keys = {"target", "prompt"}
    
    default_full_prompt = system_prefix + default_prompt + assistant_suffix
    default_valid_answer_input = (
        default_full_prompt,
        default_extras
    )
    
    try:
        dict_str = query_item            # Content inside <answer>...</answer>
        dict_str_fixed = fix_multiline_string(dict_str)
        answer_input = ast.literal_eval(dict_str_fixed)
        if not isinstance(answer_input, dict) or not required_keys.issubset(answer_input.keys()):
            return default_valid_answer_input
        prompt_text = str(answer_input.get('prompt')).strip()
        full_prompt = system_prefix + prompt_text + assistant_suffix
        answer_input['prompt'] = full_prompt
        answer_input['target'] = str(float(answer_input.get('target'))) # don't force to be an int unless for countdown
        answer_input['answer'] = answer_input['target']
        answer_input['nums'] = [int(x) for x in answer_input.get('nums', [])]

        answer_input['valid_prompt'] = True
        return (full_prompt, answer_input)
    except Exception as e:
        return default_valid_answer_input
    

extract_answer_input(query_item)

('A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\nCalculate the sum of the first 200 prime numbers Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.\nAssistant: <think>',
 {'prompt': 'A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then 

In [5]:
# Check the matching

''' Generate VLLM Raw Output: {'response': " Here are the numbers provided: [1, 2, 3]. The goal is to create an equation that equals 6 using only these three numbers, each number used once, and basic arithmetic operations (+, -, *, /).\n\nStep 1: Identify the possible combinations.\n1. Trying multiplication: (1 * 2) * 3 = 2 * 3 = 6. This seems to be one way to get the result, but we need to ensure that each number is used only once.\n\nStep 2: Check for other combinations using division.\n1. Trying division: (1 / 2) * 3 = 0.5 * 3 = 1.5, which is not equal to 6. This doesn't work.\n2. Trying another combination: 2 * 3 = 6. But we need to use all three numbers in the equation.\n\nStep 3: Check for other combinations using addition and subtraction.\n1. Trying subtraction: (1 + 2) - 3 = 3 - 3 = 0, which is not equal to 6. This doesn't work.\n2. Trying the reverse: (1 - 2) + 3 = -1 + 3 = 2, which is also not equal to 6. This doesn't work either.\n\nSo, none of the combinations using addition, subtraction, or division seem to work. Therefore, the only solution is the equation found in Step 1.</think>\n\n<answer> (1 * 2) * 3 </answer>", 'iscorrect': False, 'stop_reason': 'stop', 'final_answer': '(1 * 2) * 3', 'repeat_score': 0.011156459818955309, 'reflection_pattern_score': 0}
2025-03-03 18:26:26.804 | DEBUG    | orz.ppo.trainer:make_experience:1087 -     Extras: {'target': 6, 'nums': [1, 2, 3], 'valid_prompt': False}'''

final_answer = '(1 * 2) * 3'
extra = {'target': 6, 'nums': [1, 2, 3], 'valid_prompt': False}

In [5]:
equation = final_answer

import re
import ast
used_numbers = [int(n) for n in re.findall(r'\d+', equation)]
numbers = ast.literal_eval(str(extra["nums"]))

In [6]:
sorted(used_numbers) != sorted(numbers)

False

In [7]:
allowed_pattern = r'^[\d+\-*/().\s]+$'

In [8]:
re.match(allowed_pattern, equation)

<re.Match object; span=(0, 11), match='(1 * 2) * 3'>

In [9]:
result = eval(equation, {"__builtins__": None}, {})

In [10]:
result

6

In [12]:
abs(float(result) - float(extra["target"]))

0.0

# Checking extract answer dict

In [1]:
query_item = "{\n    'prompt': 'What is 1 + 2?',\n    'target': 3,\n    'solution': 'To solve 1 + 2, we add the two numbers together. Starting with the number 1, adding another 2 gives us 3. Therefore, the answer is 3.'\n}"

In [2]:
import ast
import re
def fix_multiline_string(dict_str):
    pattern = r"('prompt':\s*')(.*?)(')"
    def replacer(match):
        start, content, end = match.groups()
        fixed_content = content.replace('\n', '\\n')
        return f"{start}{fixed_content}{end}"
    return re.sub(pattern, replacer, dict_str, flags=re.DOTALL)

def extract_answer_input(query_item):
    system_prefix = "A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\n"
    assistant_suffix = " Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.\nAssistant: <think>" # Give example final answer to help policy model understand output better

    # TODO: adjust these default values based on domain / dataset! Pull from example final answer from prompt
    default_prompt = "What is 1 + 2?"
    default_extras = {'target': 3, 'valid_prompt': False, 'answer': 3, 'diverse': False}
    required_keys = {"target", "prompt"}
    
    default_full_prompt = system_prefix + default_prompt + assistant_suffix
    default_valid_answer_input = (
        default_full_prompt,
        default_extras
    )
    
    try:
        print('hi')
        dict_str = query_item            # Content inside <answer>...</answer>
        dict_str_fixed = fix_multiline_string(dict_str)
        answer_input = ast.literal_eval(dict_str_fixed)
        print(answer_input)
        if not isinstance(answer_input, dict) or not required_keys.issubset(answer_input.keys()):
            return default_valid_answer_input
        print('a')
        prompt_text = str(answer_input.get('prompt')).strip()
        full_prompt = system_prefix + prompt_text + assistant_suffix
        answer_input['prompt'] = full_prompt
        answer_input['target'] = str(float(answer_input.get('target')))
        answer_input['answer'] = answer_input['target']
        answer_input['nums'] = [int(x) for x in answer_input.get('nums', [])]

        answer_input['valid_prompt'] = True

        # Diversity reward # TODO: can be improved for matching and ensuring that the prompt AND target are not in self.history query section
        answer_input['diverse'] = True
        # if len(self.history) >= 6:
        #     last_query_response, last_query_reward, last_query_avg_success, last_query_final_answer = self.history[-6]
        #     if prompt_text in last_query_final_answer:
        #         answer_input['diverse'] = False
        # if len(self.history) >= 4:
        #     last_query_response, last_query_reward, last_query_avg_success, last_query_final_answer = self.history[-4]
        #     if prompt_text in last_query_final_answer:
        #         answer_input['diverse'] = False
        # if len(self.history) >= 2:
        #     last_query_response, last_query_reward, last_query_avg_success, last_query_final_answer = self.history[-2]
        #     if prompt_text in last_query_final_answer:
        #         answer_input['diverse'] = False

        return (full_prompt, answer_input)
    except Exception as e:
        print(e)
        return default_valid_answer_input

In [3]:
extract_answer_input(query_item)

hi
{'prompt': 'What is 1 + 2?', 'target': 3, 'solution': 'To solve 1 + 2, we add the two numbers together. Starting with the number 1, adding another 2 gives us 3. Therefore, the answer is 3.'}
a


('A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\nWhat is 1 + 2? Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.\nAssistant: <think>',
 {'prompt': 'A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.

In [4]:
query_item = r'''{
    'prompt': 'Consider the following system of linear equations in two variables:
                        3x + y = b
                        x - 2y = c

Determine whether the system has a unique solution, no solution, or infinitely many solutions based on the values of b and c. Provide the solution if the system has a unique solution.
           Format your solution as a modified equation where A, B, C, and D represent integers such that the equation represents a non-ambiguous value.
',
    'target': 3.0,
    'solution': 'The modified equation is 2x + 3y = (6b + 15 - 4c) / 5, where A = 2, B = 3, C = (6b + 15 - 4c) / 5, and D is not relevant.'
}'''


import re, ast

def fix_multiline_string(dict_str):
    # Match keys 'prompt' or 'solution' and capture their values
    pattern = r"((?:'prompt'|\"prompt\"|'solution'|\"solution\")\s*:\s*)(['\"])(.*?)(\2)"
    def replacer(match):
        start, quote, content, _ = match.groups()
        fixed_content = content.replace('\n', '\\n')
        return f"{start}{quote}{fixed_content}{quote}"
    return re.sub(pattern, replacer, dict_str, flags=re.DOTALL)

def extract_answer_input(query_item):
    system_prefix = "A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\n"
    assistant_suffix = " Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.\nAssistant: <think>" # Give example final answer to help policy model understand output better

    # TODO: adjust these default values based on domain / dataset! Pull from example final answer from prompt
    default_prompt = "What is 1 + 2?"
    default_extras = {'target': 3, 'valid_prompt': False, 'answer': 3, 'diverse': False}
    required_keys = {"target", "prompt"}
    
    default_full_prompt = system_prefix + default_prompt + assistant_suffix
    default_valid_answer_input = (
        default_full_prompt,
        default_extras
    )
    
    try:
        dict_str = query_item            # Content inside <answer>...</answer>
        dict_str_fixed = fix_multiline_string(dict_str)
        answer_input = ast.literal_eval(dict_str_fixed)
        if not isinstance(answer_input, dict) or not required_keys.issubset(answer_input.keys()):
            return default_valid_answer_input
        prompt_text = str(answer_input.get('prompt')).strip()
        full_prompt = system_prefix + prompt_text + assistant_suffix
        answer_input['prompt'] = full_prompt
        answer_input['target'] = str(float(answer_input.get('target'))) # don't force to be an int unless for countdown
        answer_input['answer'] = answer_input['target']
        answer_input['nums'] = [int(x) for x in answer_input.get('nums', [])]

        answer_input['valid_prompt'] = True
        return (full_prompt, answer_input)
    except Exception as e:
        return default_valid_answer_input
    

extract_answer_input(query_item)

('A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer.\nThe reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, i.e. your response format should be: <think> reasoning process here </think>\n\n<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.\nThis is the problem:\nConsider the following system of linear equations in two variables:\n                        3x + y = b\n                        x - 2y = c\n\nDetermine whether the system has a unique solution, no solution, or infinitely many solutions based on the values of b and c. Provide the solution if the system has a unique solution.\n           Format your solution as a modified equation where A, B, C, and D represent integers such that the eq

In [9]:
response = r''' To generate a task that fulfills the generation criteria, let's break down the creation of each part of the dictionary:

1. Prompt: It should be a concise but detailed math problem that is challenging yet tractable. It should be designed to encourage deeper and longer thinking from the Policy Assistant.

2. Target: It should be an integer or float that represents the correct final answer to the given math problem.

3. Solution: It should present a detailed and logical reasoning process which would lead to calculating the target value.

Here's an example of a new task that follows the given criteria:

<think>
First, let's think about a math problem that's about percentage increase. The problem should involve three digits number and a percentage increase.

Step 1: Find the difference between a three-digit number and the result of this number when increased by a percentage.

Let's consider the problem of "If 25% of a three-digit number is 75, what is the original number?".

Answer: If 25% of a three-digit number is 75, then the original number is 75 / 0.25 = 300. 

So, our solution would be: "If 25% of a three-digit number is 75, what is the original number? The original number is 300."
</think>
<answer>
{
    "prompt": "If 25% of a three-digit number is 75, what is the original number?",
    "target": 300,
    "solution": "If 25% of a three-digit number is 75, then the original number is 75 / 0.25 = 300."
}
</answer>'''

if not response.startswith("<think>"):
    response = "<think>" + response

# Check if the format is correct # allowing zero or more newlines in between </think> and <answer> because of Qwen formatting for now, which can be fixed via SFT and is NOT a focus of training
regex = r"^<think>([^<]*(?:<(?!/?think>)[^<]*)*)<\/think>\s*<answer>([\s\S]*)<\/answer>$"

match = re.search(regex, response, re.DOTALL) 
# if the format is not correct, reward is 0, or the completion doesn't end in </answer>
if match is None or \
    len(match.groups()) != 2 \
    or not response.endswith("</answer>"):
    print("Format is incorrect")

# Extract the "answer" part from the completion
answer = match.group(2).strip()
print(f"Extracted answer: {answer}")

Format is incorrect


AttributeError: 'NoneType' object has no attribute 'group'